In [1]:
import os
import requests
from typing import Type
from langchain.chat_models import ChatOpenAI
from langchain.tools import BaseTool
from pydantic import BaseModel, Field
from langchain.agents import initialize_agent, AgentType
from langchain.utilities import DuckDuckGoSearchAPIWrapper

llm = ChatOpenAI(temperature=0.1, model_name="gpt-3.5-turbo-1106")

alpha_vantage_api_key = os.environ.get("ALPHA_VANTAGE_API_KEY")


class StockMarketSymbolSearchToolArgsSchema(BaseModel):
    query: str = Field(
        description="The query you will search for.Example query: Stock Market Symbol for Apple Company"
    )


class StockMarketSymbolSearchTool(BaseTool):
    name = "StockMarketSymbolSearchTool"
    description = """
        Use this tool to find the stock market symbol for a company.
        It takes a query as an argument.
    """
    args_schema: Type[
        StockMarketSymbolSearchToolArgsSchema
    ] = StockMarketSymbolSearchToolArgsSchema

    def _run(self, query):
        ddg = DuckDuckGoSearchAPIWrapper()
        return ddg.run(query)


class CompanyOverviewArgsSchema(BaseModel):
    symbol: str = Field(
        description="Stock symbol of the company.Example: AAPL,TSLA",
    )


class CompanyOverviewTool(BaseTool):
    name = "CompanyOverview"
    description = """
        Use this to get an overview of the financials of the company.
        You should enter a stock symbol.
    """
    args_schema: Type[CompanyOverviewArgsSchema] = CompanyOverviewArgsSchema

    def _run(self, symbol):
        r = requests.get(
            f"https://www.alphavantage.co/query?function=OVERVIEW&symbol={symbol}&apikey={alpha_vantage_api_key}"
        )
        return r.json()


class CompanyIncomeStatementTool(BaseTool):
    name = "CompanyIncomeStatement"
    description = """
        Use this to get the income statement of a company.
        You should enter a stock symbol.
    """
    args_schema: Type[CompanyOverviewArgsSchema] = CompanyOverviewArgsSchema

    def _run(self, symbol):
        r = requests.get(
            f"https://www.alphavantage.co/query?function=INCOME_STATEMENT&symbol={symbol}&apikey={alpha_vantage_api_key}"
        )
        return r.json()["annualReports"]


class CompanyStockPerformanceTool(BaseTool):
    name = "CompanyStockPerformance"
    description = """
        Use this to get the daily performance of a company stock.
        You should enter a stock symbol.
    """
    args_schema: Type[CompanyOverviewArgsSchema] = CompanyOverviewArgsSchema

    def _run(self, symbol):
        r = requests.get(
            f"https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol={symbol}&apikey={alpha_vantage_api_key}"
        )
        return r.json()


agent = initialize_agent(
    llm=llm,
    verbose=True,
    agent=AgentType.OPENAI_FUNCTIONS,
    handle_parsing_errors=True,
    tools=[
        StockMarketSymbolSearchTool(),
        CompanyOverviewTool(),
        CompanyIncomeStatementTool(),
        CompanyStockPerformanceTool(),
    ],
)

prompt = "Give me financial information on Cloudflare's stock, considering its financials, income statements and stock performance help me analyze if it's a potential good investment."

agent.invoke(prompt)



> Entering new AgentExecutor chain...

Invoking: `StockMarketSymbolSearchTool` with `{'query': 'Stock Market Symbol for Cloudflare Company'}`




/Users/minjaelee/Desktop/3_personal_projects/Gpt/fullstack-gpt/env/lib/python3.11/site-packages/langchain/utilities/duckduckgo_search.py:43: UserWarning: DDGS running in an async loop. This may cause errors. Use AsyncDDGS instead.
  with DDGS() as ddgs:


Stock analysis for Cloudflare Inc (NET:New York) including stock price, stock chart, company news, key statistics, fundamentals and company profile. Cloudflare, Inc. - Stock Quote & Chart Stock Quote & Chart Stock Quote NYSE: NET 80.18 Jan 26, 2024 4:00 PM ET Today's Change -1.68 (-2.05%) Today's Volume 2,286,155 Today's Open 81.24 Previous Close 81.86 Intraday High 82.50 Intraday Low 79.80 52 Week High 87.17 52 Week Low 39.90 Pricing delayed by 20 minutes. Last Updated Jan 26, 2024 4:00 PM ET Cloudflare, Inc. is a global cloud services provider that delivers a range of services to businesses of all sizes and in all geographies. The Company's network serves as a scalable, unified control plane to deliver security, performance, and reliability across on-premise, hybrid, cloud, and software-as-a-service (SaaS) applications. Its integrated suite of products consists of solutions for an ... Cloudflare, Inc. is a global cloud services provider that delivers a range of services to businesses

{'input': "Give me financial information on Cloudflare's stock, considering its financials, income statements and stock performance help me analyze if it's a potential good investment.",
 'output': "Here's the financial information for Cloudflare Inc. (NET):\n\n1. Company Overview:\n   - Ticker Symbol: NET\n   - Description: CloudFlare, Inc. operates a cloud platform that offers a range of network services to companies around the world. The company is headquartered in San Francisco, California.\n   - Market Capitalization: $27,751,570,000\n   - Sector: Technology\n   - Industry: Services-Prepackaged Software\n   - Fiscal Year End: December\n   - Latest Quarter: 2023-09-30\n   - Analyst Target Price: $77.49\n\n2. Financial Performance:\n   - Revenue (TTM): $1,208,972,000\n   - Gross Profit (TTM): $742,631,000\n   - EBITDA: -$81,727,000\n   - EPS (TTM): -$0.61\n   - Revenue Growth (YoY): 32.2%\n   - Profit Margin: -16.7%\n   - Operating Margin (TTM): -11.7%\n   - Return on Assets (TTM): 